In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as BS
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from IPython.core import display as ICD


------------------------------------------------------------------------------------------------------------------------------
Q1 & Q2 : Scrape the following details of each product listed in first 3 pages
of your search results and save it in a dataframe and csv. In case if any product vertical has
less than 3 pages in search results then scrape all the products available under that product
vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of
Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details"
and “Product URL”. In case, if any of the details are missing for any of the product then
replace it by “-“.

------------------------------------------------------------------------------------------------------------------------------

In [2]:
def find_products(url):
    driver = webdriver.Edge("msedgedriver.exe")
    driver.get(url)
    
    time.sleep(0.5)
    
    vertical = input("Enter any keyword you want to search in Amazon website : ")
    if vertical is None:
        print("Sorry Try Again..!!")
        
    driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']").send_keys(vertical)
    driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']").submit()
    driver.implicitly_wait(2)

    brand_names = []
    products =[]
    ratings = []
    no_of_ratings = []
    prices = []
    return_or_exchange = []
    expected_delivery = []
    availability = []
    product_urls = []
    others = []
   

    page_urls = [j.get_attribute('href') for j in driver.find_elements_by_xpath("//ul[@class='a-pagination']/li/a")]
    
    for url in page_urls[0:3]:
        driver.get(url)
        
        product_urls = [i.get_attribute('href') for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")]
        
        for each_product in product_urls:
            driver.get(each_product)
            
            try:
                title = driver.find_element_by_xpath("//span[@id='productTitle']")
                products.append(title.text)
            except NoSuchElementException:
                products.append("-")
            
            
            try:
                brand_name = driver.find_element_by_xpath("//a[@id='bylineInfo']")
                brand_names.append(brand_name.text)
            except NoSuchElementException:
                brand_names.append(" - ")
            
            

            try:
                rating = driver.find_element_by_xpath("//*[@id='acrPopover']/span[1]/a/i[1]/span")
                ratings.append(rating.get_attribute('innerHTML'))
            except NoSuchElementException:
                    ratings.append(" - ")
                    
            try:
                number_of_rating = driver.find_element_by_xpath("//*[@id='acrCustomerReviewText']")
                no_of_ratings.append(number_of_rating.text)
            except NoSuchElementException:
                no_of_ratings.append(" - ")
                

            try:
                price = driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
                prices.append(price.text)
            except NoSuchElementException:
                    prices.append(" - ")
            
            try:
                returns = driver.find_element_by_xpath("//*[@id='icon-farm-container']/div/div/span/div[2]/a")
                return_or_exchange.append(returns.text)
            except NoSuchElementException:
                return_or_exchange.append(" - ")

                
                    
            try:
                delivery = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
                expected_delivery.append(delivery.text)
            except NoSuchElementException:
                    expected_delivery.append(" - ")
            
            try:
                stock = driver.find_element_by_xpath("//div[@id='availability']/span")
                availability.append(stock.get_attribute('innerHTML').replace("\n",""))
            except NoSuchElementException:
                availability.append(" - ")
                    
            try:
                other_detail = driver.find_elements_by_xpath("//*[@id='feature-bullets']/ul/li/span")
                details = ''
                for o in other_detail:
                    details += str(o.get_attribute('innerHTML').replace("\n"," "))+(';')
                others.append(details)
            except NoSuchElementException:
                others.append("-")

    product_info = pd.DataFrame({   'Title' : products,
                                    'Brand' : brand_names,
                                    'Rating ' : ratings,
                                    'Number of Ratings' : no_of_ratings,
                                    'Price' : prices,
                                    'Return or Exchange Policy' : return_or_exchange,
                                    'Expected Delivery' : expected_delivery,
                                    'Availability' : availability,
                                    'Other Details' : others
                                })
    time.sleep(0.5)
    driver.close()
    return product_info 


df = find_products("https://www.amazon.in/")
df

Enter any keyword you want to search in Amazon website : guitar


,Title,Brand,Rating,Number of Ratings,Price,Return or Exchange Policy,Expected Delivery,Availability,Other Details
0,Medellin MED-BLU-C Linden Wood Acoustic Guitar,Brand: Medellin,3.8 out of 5 stars,248 ratings,-,10 Days Replacement,"Friday, Feb 12",In stock.,Material: Wood ; Colour: Blue ; Acoustic Gu...
1,Intern Made in India Acoustic Guitar package. ...,Visit the Intern Store,-,-,"₹ 5,298.00",10 Days Replacement,"Thursday, Feb 11",Only 1 left in stock (more on the way).,Acoustic Guitar package. Made in India 41 inc...
2,Intern Made in India Acoustic Guitar package. ...,Visit the Intern Store,1.0 out of 5 stars,1 rating,"₹ 4,702.00",10 Days Replacement,"Thursday, Feb 11",In stock.,Acoustic Guitar package. Made in India 39 inc...
3,Intern Made in India Acoustic Guitar package. ...,Visit the Intern Store,-,-,"₹ 4,587.00",10 Days Replacement,"Thursday, Feb 11",In stock.,Acoustic Guitar package. Made in India 39 inc...
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the JUAREZ Store,3.8 out of 5 stars,"7,203 ratings","₹ 2,499.00",10 Days Replacement,"Tuesday, Feb 9",In stock.,"Black Glossy Finish, Number of Frets: 18, Aco..."
...,...,...,...,...,...,...,...,...,...
183,Ibanez Acoustic Guitar MD39C Right Handed Cuta...,Brand: Ibanez,4.2 out of 5 stars,9 ratings,"₹ 7,999.00",10 Days Replacement,Feb 10 - 11,In stock.,Type - Acoustic Guitar with cutaway Model - M...
184,SG Musical - 12MM Electric Guitar Gig Bag,Visit the SG Musical Store,-,-,₹ 399.00,10 Days Replacement,"Saturday, Feb 13",In stock.,Brand - SG Musical ; Quality Product ; Mate...
185,Music Gallery GUITAR WALL MOUNT WITH FITTING +...,Brand: Music Gallery,-,-,₹ 390.00,7 Days Replacement,Feb 14 - 15,In stock.,Wall bracket new design designed for to hang ...
186,Juarez Arpéggio 41 Inch Semi-Acoustic Guitar K...,Visit the JUAREZ Store,-,-,"₹ 5,392.00",10 Days Replacement,"Tuesday, Feb 9",In stock.,"Thin Line Sleek Body, Spruce Top, Glossy fini..."


-------------------------------------------------------------------------------------------------------------------------------
3. Write a python program to access the search bar and search button on images.google.com and
scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

-------------------------------------------------------------------------------------------------------------------------------

In [3]:
def fetch_image_urls(image,url):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)    

    # build the google query
    search_url = url

    # load the page
    driver = webdriver.Edge("msedgedriver.exe")
    driver.get(search_url)

    input_search = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//input[@type='text']")))
    input_search.send_keys(image)
    input_search.submit()
    driver.implicitly_wait(2)

    image_urls = []
    image_count = 0
    results_start = 0
    max_links_to_fetch = 100
    while image_count < max_links_to_fetch:
        scroll_to_end(driver)

        # get all image thumbnail results
        thumbnail_results = driver.find_elements_by_css_selector("img.rg_i.Q4LuWd")
        number_results = len(thumbnail_results)


        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(1)
            except Exception:
                continue

            # extract image urls    
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if len(image_urls) != 100:
                    if actual_image.get_attribute('src'): #and 'http' in actual_image.get_attribute('src'):
                        image_urls.append(actual_image.get_attribute('src'))

            image_count = len(image_urls)

        if len(image_urls) == max_links_to_fetch:
            break
        else:
            print("Found:", len(image_urls),"links for ", image, ", looking for more ...")
            time.sleep(30)
            load_more_button = driver.find_element_by_cpath("//input[@class='mye4qd']")
            if load_more_button:
                load_more_button.click()

        # move the result startpoint further down
        results_start = len(thumbnail_results)
    driver.close()
        

    return image_urls


url = "https://images.google.com/?gws_rd=ssl"
inputs = ["fruits","cars","Machine Learning"]
url_list = pd.DataFrame()

for each in inputs:
    url_list [each] = fetch_image_urls(each,url)
    

print("Url list of images : ")
url_list   
    

Url list of images : 


,fruits,cars,Machine Learning
0,https://i.ytimg.com/vi/PY2OxP_jMZ0/maxresdefau...,https://specials-images.forbesimg.com/imageser...,https://thumbor.forbes.com/thumbor/fit-in/1200...
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",https://etimg.etb2bimg.com/photo/73002933.cms,https://miro.medium.com/max/2400/1*c_fiB-YgbnM...
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
98,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...


-------------------------------------------------------------------------------------------------------------------------------
4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page.
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”,
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe
and CSV.

-------------------------------------------------------------------------------------------------------------------------------

In [9]:
def smartphones(url):
    driver = webdriver.Edge("msedgedriver.exe")
    driver.get(url)
    time.sleep(2)
    
    #close pop_up
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
    
    smartphone = 'Google Pixel 4A'
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//*[@placeholder='Search for products, brands and more']"))).send_keys(smartphone)
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[@class='L0Z3Pu']"))).click()
    
    brands = []
    products = []
    colour = []
    ram = []
    storage = []
    primary_cam = []
    secondary_cam = []
    display_size = []
    display_resolution = []
    processor = []
    processor_cores = []
    battery_capacity = []
    prices = []
    product_urls = []

    

    current_url = driver.current_url
    driver.get(current_url)
    
        
    product_url = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
    for each in product_url:
        product_urls.append(each.get_attribute('href'))
        
    for url in product_urls:
        driver.get(url)
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']"))).click()
        
        title = driver.find_element_by_xpath("//*[@id='container']/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span")
        try:
            brands.append(title.text.split(" ")[0])
            products.append(title.text)
        except NoSuchElementException:
            brands.append(" -- ")
            products.append()
            
            
        
        try:
            price = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
            prices.append(price.text)
        except NoSuchElementException:
            prices.append(" -- ")
            
        try:
            color = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[1]/table/tbody/tr[4]/td[2]/ul/li")
            colour.append(color.get_attribute('innerHTML'))
        except NoSuchElementException:
            colour.append(" -- ")
            
        try:
            random_memory = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[4]/table/tbody/tr[2]/td[2]/ul/li")
            ram.append(random_memory.get_attribute('innerHTML'))
        except NoSuchElementException:
            ram.append(" -- ")
            
        try:
            display = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[2]/table/tbody/tr[1]/td[2]/ul/li")
            display_size.append(display.get_attribute('innerHTML'))
        except NoSuchElementException:
            display_size.append(" -- ")
            
        try:
            resolution = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[2]/table/tbody/tr[2]/td[2]/ul/li")
            display_resolution.append(resolution.get_attribute('innerHTML'))
        except NoSuchElementException:
            display_resolution.append(" -- ")
            
        try:
            rom = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[4]/table/tbody/tr[1]/td[2]/ul/li")
            storage.append(rom.get_attribute('innerHTML'))
        except NoSuchElementException:
            storage.append(" -- ")
            
        try:
            primary = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[5]/table/tbody/tr[2]/td[2]/ul/li")
            primary_cam.append(primary.get_attribute('innerHTML'))
        except NoSuchElementException:
            primary_cam.append(" -- ")
            
        try:
            secondary = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[5]/table/tbody/tr[5]/td[2]/ul/li")
            secondary_cam.append(secondary.get_attribute('innerHTML'))
        except NoSuchElementException:
            secondary_cam.append(" -- ")
            
        try:
            processors = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[3]/table/tbody/tr[2]/td[2]/ul/li")
            processor.append(processors.get_attribute('innerHTML'))
        except NoSuchElementException:
            processor.append(" -- ")
        
        try:
            cores = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[3]/table/tbody/tr[3]/td[2]/ul/li")
            processor_cores.append(cores.text.split(",")[0])
        except NoSuchElementException:
            processor_cores.append(" -- ")
            
        try:
            battery = driver.find_element_by_xpath("//div[@class='_3dtsli']/div/div/div[8]/table/tbody/tr[1]/td[2]/ul/li")
            battery_capacity.append(battery.text)
        except NoSuchElementException:
            battery_capacity.append(" -- ")
            
        
    smart_phone = pd.DataFrame({'Smartphone Name': products,
                                'Brand': brands,
                                'Color' : colour,
                                'RAM': ram,
                                'Storage (ROM) ': storage,
                               'Display Size': display_size,
                               'Display Resolution' : display_resolution,
                               'Primary Camera' : primary_cam,
                               'Secondary Camera' : secondary_cam,
                               'Processor' : processor,
                               'Processor Cores' : processor_cores,
                               'Battery Capacity': battery_capacity,
                               'Prices' : prices,
                               'Product URLs' : product_urls})   
    
    driver.close()
    return smart_phone
    
df = smartphones("https://www.flipkart.com/")
df.to_csv("smart_phone_info.csv")
df 

,Smartphone Name,Brand,Color,RAM,Storage (ROM),Display Size,Display Resolution,Primary Camera,Secondary Camera,Processor,Processor Cores,Battery Capacity,Prices,Product URLs
0,"Google Pixel 4a (Just Black, 128 GB) (6 GB RAM)",Google,Just Black,6 GB,128 GB,14.76 cm (5.81 inch),2340 x 1080 Pixels,12.2MP Rear Camera,8MP Front Camera,Qualcomm Snapdragon 730G,Octa Core,Yes,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,"Google Pixel 3a (Clearly White, 64 GB) (4 GB ...",Google,Clearly White,4 GB,64 GB,14.22 cm (5.6 inch),2220 x 1080 pixels,12.2MP Rear Camera,8MP Front Camera,Qualcomm Snapdragon 670,Octa Core,3000 mAh,"₹30,999",https://www.flipkart.com/google-pixel-3a-clear...
2,"Google Pixel 3 XL (Clearly White, 64 GB) (4 G...",Google,Clearly White,4 GB,64 GB,16.0 cm (6.3 inch),2960 x 1440 pixels,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,Qualcomm Snapdragon 845 64-bit,Octa Core,3430 mAh,"₹57,999",https://www.flipkart.com/google-pixel-3-xl-cle...
3,"Google Pixel 3 (Clearly White, 128 GB) (4 GB ...",Google,Clearly White,4 GB,128 GB,13.97 cm (5.5 inch),2160 x 1080 pixels,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,Qualcomm Snapdragon 845 64-bit,Octa Core,2915 mAh,"₹80,000",https://www.flipkart.com/google-pixel-3-clearl...
4,"Google Pixel 3 (Not Pink, 128 GB) (4 GB RAM)",Google,Not Pink,4 GB,128 GB,13.97 cm (5.5 inch),2160 x 1080 pixels,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,Qualcomm Snapdragon 845 64-bit,Octa Core,2915 mAh,"₹80,000",https://www.flipkart.com/google-pixel-3-not-pi...
5,"Google Pixel 3 (Clearly White, 64 GB) (4 GB RAM)",Google,Clearly White,4 GB,64 GB,13.97 cm (5.5 inch),2160 x 1080 pixels,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,Qualcomm Snapdragon 845 64-bit,Octa Core,2915 mAh,"₹71,000",https://www.flipkart.com/google-pixel-3-clearl...
6,"Google Pixel 3 (Just Black, 64 GB) (4 GB RAM)",Google,Just Black,4 GB,64 GB,13.97 cm (5.5 inch),2160 x 1080 pixels,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,Qualcomm Snapdragon 845 64-bit,Octa Core,2915 mAh,"₹71,000",https://www.flipkart.com/google-pixel-3-just-b...
7,"Google Pixel 3 (Just Black, 128 GB) (4 GB RAM)",Google,Just Black,4 GB,128 GB,13.97 cm (5.5 inch),2160 x 1080 pixels,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,Qualcomm Snapdragon 845 64-bit,Octa Core,2915 mAh,"₹80,000",https://www.flipkart.com/google-pixel-3-just-b...
8,"Google Pixel 2 XL (Black & White, 64 GB) (4 G...",Google,Black &amp; White,4 GB,64 GB,15.24 cm (6 inch),2880 x 1440 Pixels,12.2MP Rear Camera,8MP Front Camera,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,3520 mAh,"₹45,499",https://www.flipkart.com/google-pixel-2-xl-bla...
9,"Google Pixel 2 (Kinda Blue, 64 GB) (4 GB RAM)",Google,Kinda Blue,4 GB,64 GB,12.7 cm (5 inch),1920 x 1080 Pixels,12.2MP Rear Camera,8MP Front Camera,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,2700 mAh,"₹61,000",https://www.flipkart.com/google-pixel-2-kinda-...


5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on
google maps.

In [5]:
def find_geospatial_coordinates(city):
    url = 'https://www.google.com/maps'
    driver = webdriver.Edge("msedgedriver.exe")
    driver.get(url)
    driver.maximize_window()


    search_city = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//input[@class='tactile-searchbox-input']")))
    search_city.send_keys(city)
    search_city.send_keys(Keys.ENTER)

    time.sleep(10)

    current_url = driver.current_url
    coordinates = current_url.split("@")[1].split(",")[0:2]
    print("******************************************************************")
    print("Geospatial Coordinates for city : ", city)
    print("Latitude : ",coordinates[0])
    print("Longitude: ",coordinates[1])
    print("*******************************************************************")
    driver.close()
    
city = ['Kathmandu','Bangalore','Lahore','Pennsylvania','California']
for each in city:
    find_geospatial_coordinates(each)

******************************************************************
Geospatial Coordinates for city :  Kathmandu
Latitude :  27.7090319
Longitude:  85.2911131
*******************************************************************
******************************************************************
Geospatial Coordinates for city :  Bangalore
Latitude :  12.9538477
Longitude:  77.3507301
*******************************************************************
******************************************************************
Geospatial Coordinates for city :  Lahore
Latitude :  31.4832209
Longitude:  74.0541892
*******************************************************************
******************************************************************
Geospatial Coordinates for city :  Pennsylvania
Latitude :  41.0993556
Longitude:  -79.8485319
*******************************************************************
******************************************************************
Geospatial Coordinates for c

6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in.


In [10]:
def get_report(url,month_name):
    driver = webdriver.Edge("msedgedriver.exe")
    driver.get(url)
    driver.maximize_window()

    WebDriverWait(driver,15).until(EC.element_to_be_clickable((By.XPATH,"//a[@title='https://trak.in/india-startup-funding-investment-2015/']"))).click()
    #close popup
    #WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//div[@id='dismiss-button']"))).click()

    serial_no = []
    dates = []
    startup_name = []
    industry = []
    sub_vertical = []
    location = []
    investors_names = []
    investment_types = []
    amounts = []

    month = driver.find_elements_by_xpath("//h2")
    for i in month:
        if i.text == month_name:
            id_value = i.get_attribute('class').split("-")[-1]
            
    
    path = 'tablepress-'+id_value
    #click on : Show 100 entries
    WebDriverWait(driver,15).until(EC.element_to_be_clickable((By.XPATH,"//select[@aria-controls='" + path + "']/option[4]"))).click()


    try:
        num = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[1]")
        for n in num:
            serial_no.append(n.text)
    except NoSuchElementException:
        serial_no.append("-")

    try:
        date = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[2]")
        for d in date:
            dates.append(d.text)
    except NoSuchElementException:
        dates.append("-")
    try:
        startup = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[3]")
        for s in startup:
            startup_name.append(s.text)
    except NoSuchElementException:
        startup_name.append("-")
    try:
        vertical = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[4]")
        for v in vertical:
            industry.append(v.text)
    except NoSuchElementException:
        industry.append("-")
    try:
        sub = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[5]")
        for s in sub:
            sub_vertical.append(s.text)
    except NoSuchElementException:
        sub_vertical.append("-")
    try:
        loc = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[6]")
        for l in loc:
            location.append(l.text)
    except NoSuchElementException:
        location.append("-")
    try:
        names = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[7]")
        for n in names:
            investors_names.append(n.text)
    except NoSuchElementException:
        investors_names.append("-")
    try:
        type = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[8]")
        for t in type:
            investment_types.append(t.text)
    except NoSuchElementException:
        investment_types.append("-")
    try:
        amount = driver.find_elements_by_xpath("//*[@id='" + path + "']/tbody/tr/td[9]")
        for a in amount:
            amounts.append(a.text)
    except NoSuchElementException:
        amounts.append("-")

    report = pd.DataFrame({'Serial No.' : serial_no,
                  'Date ( dd/mm/yyyy )' : dates,
                  'Startup Name' : startup_name,
                  'Industry/Vertical' : industry,
                  'Sub-Vertical' : sub_vertical, 
                  'City/Location' : location,
                  'Investor\'s Name': investors_names,
                  'Investment Type': investment_types,
                  'Amount ( In USD )': amounts})
        
    driver.close()
                                                
        
    return report
                                            

url = "https://trak.in/"
month_list = ['July, 2020', 'August, 2020','September, 2020']
df = pd.DataFrame()

for month in month_list:
    print(" ")
    print("-------------------------------------------------------------------------------------------------------------------")
    print("                               Retrieving Funding Deals for :  ", month, "            ")
    print("-------------------------------------------------------------------------------------------------------------------")
    print(" ")
    df = get_report(url,month)
    ICD.display(df)

 
-------------------------------------------------------------------------------------------------------------------
                               Retrieving Funding Deals for :   July, 2020             
-------------------------------------------------------------------------------------------------------------------
 


,Serial No.,Date ( dd/mm/yyyy ),Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor's Name,Investment Type,Amount ( In USD )
0,1,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,2,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,3,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,4,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,5,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,6,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,7,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,8,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,9,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,10,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


 
-------------------------------------------------------------------------------------------------------------------
                               Retrieving Funding Deals for :   August, 2020             
-------------------------------------------------------------------------------------------------------------------
 


,Serial No.,Date ( dd/mm/yyyy ),Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor's Name,Investment Type,Amount ( In USD )
0,1,15/08/2020,Practo,HealthTech,Health care and Wellness,Bangalore,A1A Company,Series F,"32,000,000"
1,2,13/08/2020,Medlife,E-commerce,Online Pharmacy,Bangalore,Prasid Uno Family Trust and SC Credit Fund,,"23,000,000"
2,3,13/08/2020,HungerBox,FoodTech,Online Food Delivery Service,Bangalore,"One97, Sabre Partners Trust, Pratithi Investme...",Series D1,"1,560,000"
3,4,04/08/2020,Dunzo,Hyper-local Logistics,Online Delivery Services,Bangalore,Existing Backers,In Progress,"30,000,000"
4,5,11/08/2020,Terra.do,EduTech,"Online Climate School, E-learning","Stanford, California,","Stanford Angels and Entrepreneurs (India), BEE...",Seed,"1,400,000"
5,6,12/08/2020,Classplus,EduTech,"E-learning, Online Tutoring",Noida,Falcon Edge,In Progress,"upto 15,000,000"
6,7,14/08/2020,Niyo,FinTech,Financial Services,Bangalore,Niyo Solutions Inc.,,"6,000,000"
7,8,10/08/2020,ZestMoney,FinTech,Financial Services,Bangalore,Primrose Hills Ventures,,"10,670,000"
8,9,07/08/2020,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,"16,200,000"
9,10,13/08/2020,Eduvanz,FinTech,Financial Services,Mumbai,"Sequoia India, Unitus",Series A,"5,000,000"


 
-------------------------------------------------------------------------------------------------------------------
                               Retrieving Funding Deals for :   September, 2020             
-------------------------------------------------------------------------------------------------------------------
 


,Serial No.,Date ( dd/mm/yyyy ),Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor's Name,Investment Type,Amount ( In USD )
0,1,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
1,2,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
2,3,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
3,4,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
4,5,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
5,6,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
6,7,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
7,8,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
8,9,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
9,10,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in. 

In [7]:
 def find_gaming_laptops(search_url):
        
        driver = webdriver.Edge("msedgedriver.exe")
        driver.get(search_url)

        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='search']/a"))).click()
        #input_search = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//input[@type='search']")))
        #input_search.send_keys("gaming laptop")
        #input_search.send_keys()

        driver.find_element_by_xpath("//input[@type='search']").send_keys("top 10 gaming laptops")
        driver.find_element_by_xpath("//input[@type='search']").send_keys(Keys.ENTER)
        driver.implicitly_wait(2)

        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//input[@value='toptens']"))).click()

        #pages
        product_titles = []
        weights = []
        graphic_processor = []
        os = []
        display = []
        processors = []
        memory = []
        price = []
        dimension = []
        description = []



        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='searchProduct-desc']"))).click()


        try:
            titles = driver.find_elements_by_xpath("//h3")
            for t in titles:
                product_titles.append(t.get_attribute('innerHTML'))
        except NoSuchElementException:
            product_titles.append("-")

        try:
            weight = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
            for w in weight:
                weights.append(w.get_attribute('innerHTML').split(" ")[0])
        except NoSuchElementException:
            weights.append("-")

        try:
            displays = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]")
            for d in displays:
                display.append(d.get_attribute('innerHTML'))
        except NoSuchElementException:
            display.append("-")

        try:
            processor = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]")
            for p in processor:
                processors.append(p.text)
        except NoSuchElementException:
            processors.append("-")
        try:
            operating_system = driver.find_elements_by_xpath("//div[@class='Product-specs']/div[2]/div[1]/ul/li[1]/div/div")
            for o in operating_system:
                os.append(o.text)
        except NoSuchElementException:
            os.append("-")
        try:
            gpu = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]")
            for g in gpu:
                graphic_processor.append(g.get_attribute('innerHTML'))
        except NoSuchElementException:
            graphic_processor.append("-")
        try:
            cost = driver.find_elements_by_xpath("//*[@id='summtable']/tbody/tr/td[3]")
            for c in cost:
                price.append(c.text)
        except NoSuchElementException:
            price.append("-")

        try:
            ram = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]")
            for r in ram:
                memory.append(r.text.replace("\n",""))
        except NoSuchElementException:
            memory.append("-")


        try:
            dim = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
            for d in dim:
                dimension.append(d.get_attribute('innerHTML').replace("mm",""))
        except NoSuchElementException:
            dimension.append("-")

        try:
            desc = driver.find_elements_by_xpath("//div[@class='Section-center']/p")
            for d in desc:
                 description.append(d.get_attribute('innerHTML'))
        except NoSuchElementException:
            description.append("-")

        driver.close()


        gaming_laptops = pd.DataFrame({'Product Name ': product_titles,
              'Operating System' : os,
              'Processor' : processors ,
              'Memory' : memory ,
              'Display' : display ,
              'Weight (Kg)' : weights,
              'Dimensions (mm)' : dimension,
              'Graphic Processor' : graphic_processor,
              'Description' : description,
              'Price' : price
             }) 

        return gaming_laptops
    
df = find_gaming_laptops("https://www.digit.in/")
df

,Product Name,Operating System,Processor,Memory,Display,Weight (Kg),Dimensions (mm),Graphic Processor,Description,Price
0,MSI GT76 TITAN DT 9SG,WINDOWS 10 PRO,Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,"17.3"" (UHD 3840x2160)",4.2,397 x 330 x 33~42,NVIDIA GeForce RTX 2080,Sporting a desktop graphics Intel Core i9-9900...,₹379990
1,ALIENWARE 17 AREA-51M,WINDOWS 10 PRO,Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,"17.3"" (FHD (1920 x 1080))",3.87,42 x 402.6 x 319.14,NVIDIA GeForce RTX 2080,The Alienware Area-51m was certainly the first...,₹489909
2,HP OMEN 15 2020,WINDOWS 10 HOME,Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,"15.6"" (1920 x 1080)",5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,The new HP Omen 15 comes in both 10th generati...,₹117790
3,ASUS ZEPHYRUS G14,WINDOWS 10 HOME,AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,"14"" (1920 x 1080)",1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...,₹164990
4,LENOVO LEGION Y540,WINDOWS 10 HOME,9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,"15.6"" (1920 X 1080)",2.3,365 x 260 x 25.9,NVIDIA® GeForce RTX™ 2060,The Lenovo Legion Y540 is powered by an Intel ...,₹79990
5,ASUS ROG ZEPHYRUS G GA502,WINDOWS 10 HOME,AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,"15.6"" (1920 x 1080)",2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,The Ryzen 7 3750H powered Asus ROG Zephyrus G ...,₹79990
6,ASUS ROG ZEPHYRUS S GX531,WINDOWS 10 HOME,9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,"15.6"" (1920 X 1080)",2.1,360 (W) x 268 (D) x 15.35~16.15 (H),NVIDIA® GeForce RTX™ 2070 (Max-Q),The Asus Zephyrus S (GX531) manages to get a f...,₹146000
7,MSI GT83VR 7RE TITAN SLI,WINDOWS 10 HOME 64 BIT,Intel CM238 Core i7-7820HK+CM238 7th Gen | 3.5GHz,1.5 TB SATA/64GB DDR4,"18.4"" (1920 x 1080)",5.5,458 x 339 x 69,Dual GTX1070,MSI does not have a dual GTX 1080 gaming lapto...,₹349990
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,"15.6"" (3840 x 1100)",2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...,₹268990
9,DELL G3,WINDOWS 10 HOME PLUS,8th Generation Intel® Core™ i7-8750H | NA,256 + 1 TB SSD + SATA/16GB DDR4,15.6 MP | NA,2.53,22.7 X 380 X 258,NVIDIA® GeForce® GTX,The Dell G3 is a capable entry-level gaming la...,₹61990
